Загрузка и обработка файлов

In [1]:
import os
import hashlib
from itertools import chain
import shutil
import csv
import cv2

import json
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Получим список жанров

# Укажем путь к каталогу, в котором находятся подкаталоги
directory_to_check = 'data'

# Получаем список всех подкаталогов в указанном каталоге
genres = [name for name in os.listdir(directory_to_check) if os.path.isdir(os.path.join(directory_to_check, name))]

# Выводим список жанров
print("Список жанров:")
for genres in genres:
    print(genres)

Список жанров:
anime
black metal
classical
country
disco
edm
jazz
pop
rap
reggae


In [3]:
# Проверим на дубликаты 

def find_duplicate_files(directory):
    # Словарь для хранения хешей файлов
    files_hashes = {}
    # Словарь для хранения дубликатов файлов
    duplicates = {}

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            # Вычисляем хеш файла
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()

            # Проверяем наличие хеша в словаре
            if file_hash in files_hashes:
                # Если хеш уже есть, добавляем файл в список дубликатов
                if file_hash not in duplicates:
                    duplicates[file_hash] = []
                duplicates[file_hash].append(file_path)
            else:
                # Если хеша нет, добавляем его в словарь
                files_hashes[file_hash] = file_path

    return duplicates


# Поиск дубликатов файлов
duplicate_files = find_duplicate_files(directory_to_check)

# Проверка наличия дубликатов и вывод результата
if duplicate_files:
    print("Найдены дубликаты файлов:")
    for hash_value, file_paths in duplicate_files.items():
        print(f"Хеш: {hash_value}")
        for file_path in file_paths:
            print(f" - Файл: {file_path}")
    print(f"Всего найдено дубликатов: {sum(len(file_paths) for file_paths in duplicate_files.values())}")
else:
    print("Дубликаты файлов не найдены.")

Найдены дубликаты файлов:
Хеш: 39ec26aaf7ca0f5bb2b72a48e342cc3a
 - Файл: data\anime\054883a9-b2e2-4e52-ad52-c980ea768cf0.png
Хеш: ca399abdf335eb0e6a811d606e0ad2e9
 - Файл: data\anime\16072b25-550c-43b3-9cd3-6f575ec0a0ee.png
Хеш: c47db685b68eac300e619d2b14b51178
 - Файл: data\anime\680519a3-9a87-428b-8d3b-c7ef4b005de6.png
Хеш: 8c5cd324bc7cefca90da6f0b405e159c
 - Файл: data\anime\bf9fe9d3-9393-4d02-9e21-db866297241b.png
Хеш: 3bf2b874082432442cce67ab6d6a9623
 - Файл: data\anime\e0f6f77d-b589-4370-aa16-975fa91f481e.png
Хеш: aef005dc0196afd838f4c7d269a45bd5
 - Файл: data\black metal\151a2a52-ffe7-35e1-9fa2-b03d1cd86fb7.png
Хеш: ba82031a54e57040b76c0f2b2cf5529c
 - Файл: data\classical\59e085a2-1b7f-4629-bbf5-d02244e05983.png
Хеш: 341188ec0399a1ca94543ed2def668a8
 - Файл: data\classical\8e012a89-4adf-4ac1-8837-a740d7da7d0f.png
Хеш: aab8a092ae2e66521217f9eb0b64b9de
 - Файл: data\classical\ce7a56b3-f52a-4ad0-81f1-7a5a041bcc34.png
 - Файл: data\classical\ff0f6253-00db-4472-85a9-8870ba4014b7.png


In [4]:
# Функция для удаления дубликатов из подкаталогов
def remove_duplicate_files(directory):
    files_hashes = {}

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Вычисляем хеш файла
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()

            # Проверяем наличие хеша в словаре
            if file_hash in files_hashes:
                # Если хеш уже есть, удаляем файл
                print(f"Удаляем дубликат файла: {file_path}")
                os.remove(file_path)
            else:
                # Если хеша нет, добавляем его в словарь
                files_hashes[file_hash] = file_path



# Удаление дубликатов файлов
remove_duplicate_files(directory_to_check)

Удаляем дубликат файла: data\anime\054883a9-b2e2-4e52-ad52-c980ea768cf0.png
Удаляем дубликат файла: data\anime\16072b25-550c-43b3-9cd3-6f575ec0a0ee.png
Удаляем дубликат файла: data\anime\680519a3-9a87-428b-8d3b-c7ef4b005de6.png
Удаляем дубликат файла: data\anime\bf9fe9d3-9393-4d02-9e21-db866297241b.png
Удаляем дубликат файла: data\anime\e0f6f77d-b589-4370-aa16-975fa91f481e.png
Удаляем дубликат файла: data\black metal\151a2a52-ffe7-35e1-9fa2-b03d1cd86fb7.png
Удаляем дубликат файла: data\classical\59e085a2-1b7f-4629-bbf5-d02244e05983.png
Удаляем дубликат файла: data\classical\8e012a89-4adf-4ac1-8837-a740d7da7d0f.png
Удаляем дубликат файла: data\classical\ce7a56b3-f52a-4ad0-81f1-7a5a041bcc34.png
Удаляем дубликат файла: data\classical\ff0f6253-00db-4472-85a9-8870ba4014b7.png
Удаляем дубликат файла: data\country\62b6ad6f-919d-487e-8a93-2360ad45cb52.png
Удаляем дубликат файла: data\country\c9e52ce6-963f-4891-94aa-8c3e19224d4e.png
Удаляем дубликат файла: data\country\d727b7de-bc03-41a3-82eb-0

In [5]:
# Создаем словарь, в котором ключами являются названия подкаталогов,
# а значениями - списки файлов, находящихся в этих подкаталогах. 
# Каждый файл представлен своим полным путем.

def create_directory_file_map(directory):
    file_to_label = {}

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            directory_name = os.path.basename(root)
            if directory_name != os.path.basename(directory):
                if directory_name not in file_to_label:
                    file_to_label[directory_name] = []
                file_to_label[directory_name].append(file_path)

    return file_to_label

# Создание словаря
file_to_label = create_directory_file_map(directory_to_check)

In [6]:
# Сохраним этот словарь на диск

# Убедимся, что каталог существует
if not os.path.exists(directory_to_check):
    os.makedirs(directory_to_check)

# Полный путь к файлу
file_path = os.path.join(directory_to_check, "file_to_label.json")

# Запись словаря в JSON файл
with open(file_path, 'w') as file:
    json.dump(file_to_label, file)

print(f"Словарь успешно сохранен в {file_path}")

Словарь успешно сохранен в data\file_to_label.json


In [7]:
# Вывод всех ключей и по 1 значению для каждого ключа
for key, values in file_to_label.items():
    print(f"Ключ: {key}")
    print(f"Значение(я): {values[0]}")
    print(f"Количество обложек: {len(values)}")
    print('***')
    # print()

Ключ: anime
Значение(я): data\anime\00095702-6b86-4bb1-880e-1e09e759a013.png
Количество обложек: 1184
***
Ключ: black metal
Значение(я): data\black metal\0045bdd1-6829-4ecc-82ea-99a9958d5fcb.png
Количество обложек: 449
***
Ключ: classical
Значение(я): data\classical\0049cfb2-17e7-4316-b60c-4499df770492.png
Количество обложек: 951
***
Ключ: country
Значение(я): data\country\002b7538-2c76-46f0-b4f4-896002222dae.png
Количество обложек: 1483
***
Ключ: disco
Значение(я): data\disco\00af76c0-0e01-4545-b050-8b548bc95d56.png
Количество обложек: 466
***
Ключ: edm
Значение(я): data\edm\00024487-47fe-46be-8820-e772c93966b4.png
Количество обложек: 702
***
Ключ: jazz
Значение(я): data\jazz\003dad6c-43d5-39ea-8d23-009d3eca6920.png
Количество обложек: 819
***
Ключ: pop
Значение(я): data\pop\00f71d58-7ab6-4b09-9bc0-aa5e44f74f7a.png
Количество обложек: 790
***
Ключ: rap
Значение(я): data\rap\006bb981-d9e6-4ccb-93d2-33e061f8b376.png
Количество обложек: 326
***
Ключ: reggae
Значение(я): data\reggae\0086f

In [8]:
# Подсчет общего количества значений
total_values = sum(len(files) for files in file_to_label.values())

print(f"Общее количество обложек для анализа: {total_values}")

Общее количество обложек для анализа: 7748


In [9]:
# Разделим данные для каждого класса отдельно, учитывая их пропорции.

# Создаем пустые списки для обучающих и тестовых данных
train_image_paths = []
test_image_paths = []
train_labels = []
test_labels = []

# Проходим по каждому классу в словаре file_to_label
for label, image_paths in file_to_label.items():
    # Разделяем пути к файлам для каждого класса
    train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)
    
    # Добавляем разделенные пути к файлам в соответствующие списки
    train_image_paths.extend(train_paths)
    test_image_paths.extend(test_paths)
    
    # Добавляем метку класса для каждого соответствующего пути к файлу
    train_labels.extend([label] * len(train_paths))
    test_labels.extend([label] * len(test_paths))

# Выводим размеры обучающей и тестовой выборок
print("Размер обучающей выборки:", len(train_image_paths))
print("Размер тестовой выборки:", len(test_image_paths))


Размер обучающей выборки: 6194
Размер тестовой выборки: 1554


In [10]:
# создаем файлы CSV train_data.csv и test_data.csv 
# и записываем в них данные из списков train_data и test_data. 
# Каждая строка в файле CSV будет содержать путь к изображению 
# и соответствующую метку

# Задаем имена файлов CSV для обучающих и тестовых данных
train_csv_file = "train_data.csv"
test_csv_file = "test_data.csv"

# Собираем данные в списки списков для записи в CSV
train_data = list(zip(train_image_paths, train_labels))
test_data = list(zip(test_image_paths, test_labels))

# Функция для записи данных в файл CSV
def write_data_to_csv(data, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image_path', 'label'])  # Записываем заголовки столбцов
        writer.writerows(data)

# Записываем данные в файлы CSV
write_data_to_csv(train_data, train_csv_file)
write_data_to_csv(test_data, test_csv_file)

print("Данные успешно записаны в файлы CSV:", train_csv_file, "и", test_csv_file)

Данные успешно записаны в файлы CSV: train_data.csv и test_data.csv


In [11]:
# Проверим размер изображений
def get_image_sizes(image_paths):
    image_sizes = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            height, width, _ = image.shape
            image_sizes.append((height, width))
    return image_sizes

# Пример использования:
# Предположим, что у вас есть список image_paths, содержащий пути к изображениям
# image_paths = [...] 
image_sizes = get_image_sizes(test_image_paths)

# Вывод уникальных размеров изображений
unique_sizes = np.unique(image_sizes, axis=0)
print("Уникальные размеры изображений:")
for size in unique_sizes:
    print(f"Высота: {size[0]}, Ширина: {size[1]}")

Уникальные размеры изображений:
Высота: 300, Ширина: 300


In [12]:
# # Изменим размер изображений
# def resize_images_in_directory(input_dir, target_size=(224, 224)):
#     # Проходим по всем элементам в директории
#     for item in os.listdir(input_dir):
#         # Получаем полный путь к текущему элементу
#         item_path = os.path.join(input_dir, item)
        
#         # Если текущий элемент является файлом, обрабатываем его
#         if os.path.isfile(item_path):
#             # Проверяем, является ли файл изображением
#             _, ext = os.path.splitext(item_path)
#             if ext.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
#                 # Читаем изображение
#                 image = cv2.imread(item_path)
                
#                 if image is not None:  # Проверяем, удалось ли прочитать изображение
#                     # Изменяем размер изображения
#                     resized_image = cv2.resize(image, target_size)
                    
#                     # Сохраняем измененное изображение в той же директории
#                     cv2.imwrite(item_path, resized_image)
#                     print(f"Изображение {item_path} успешно изменено и сохранено.")
#                 else:
#                     print(f"Ошибка чтения изображения {item_path}. Пропускаем.")
        
#         # Если текущий элемент является директорией, рекурсивно вызываем эту функцию для нее
#         elif os.path.isdir(item_path):
#             # Рекурсивно вызываем функцию для обработки поддиректории
#             resize_images_in_directory(item_path, target_size)

# # Указываем путь к директории, содержащей изображения, которые нужно изменить
# input_directory = "data"

# # Изменяем размер всех изображений в директории и ее поддиректориях
# resize_images_in_directory(input_directory)


In [13]:
# # Укажите путь к основному каталогу, содержащему подкаталоги
# main_directory = input_directory

# # Перебираем все подкаталоги
# for root, dirs, files in os.walk(main_directory):
#     # Если в каталоге нет файлов, переходим к следующему каталогу
#     if not files:
#         continue
    
#     # Сортируем список файлов по алфавиту
#     files.sort()
    
#     # Оставляем только первые 100 файлов
#     files_to_keep = files[:100]
    
#     # Удаляем оставшиеся файлы
#     for file_name in files[100:]:
#         file_path = os.path.join(root, file_name)
#         os.remove(file_path)

In [14]:
# Создадим внутри каталога data подкаталоги train и test. 
# Затем скопируем файлы из обучающей и тестовой выборок в соответствующие подкаталоги.

# Создание подкаталогов train и test в каталоге data, если они не существуют
# train_directory = os.path.join(directory_to_check, 'train')
# test_directory = os.path.join(directory_to_check, 'test')
# for directory in [train_directory, test_directory]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# # Копирование файлов обучающей выборки в каталог train
# for src_path in train_image_paths:
#     filename = os.path.basename(src_path)
#     dst_path = os.path.join(train_directory, filename)
#     shutil.copy(src_path, dst_path)

# # Копирование файлов тестовой выборки в каталог test
# for src_path in test_image_paths:
#     filename = os.path.basename(src_path)
#     dst_path = os.path.join(test_directory, filename)
#     shutil.copy(src_path, dst_path)

# print("Данные успешно записаны в каталог data.")

## Вывод по этапу.   
Посмотрели данные, удалили дубликаты.  
Общее количество обложек для анализа: 7748  
Создали словарь, в которм ключи - это жанры, значения - пути к файлам обложек.  
Разделили выборку на тренировочную и тестовую в пропорции 80 к 20.  
При необходимости можем разделить данные на тренировочные и тестовые по отдельным подкаталогам. 
Пока не знаю, есть ли такая необходимость.  